In [1]:
import pandas as pd
import requests
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from urllib.parse import quote_plus

load_dotenv()

ninja_key = os.getenv('NINJA_API_KEY')
user = quote_plus(os.getenv('REDSHIFT_USER'))
password = quote_plus(os.getenv('REDSHIFT_PASSWORD'))
host = os.getenv('REDSHIFT_HOST')
port = os.getenv('REDSHIFT_PORT')
database = os.getenv('REDSHIFT_DB')

In [2]:
def get_crypto_symbols():
    symbols = 'https://api.api-ninjas.com/v1/cryptosymbols'
    response = requests.get(symbols, headers={'X-Api-Key': ninja_key})
    if response.status_code == requests.codes.ok:
        return json.loads(response.text)['symbols']
    else:
        print("Error:", response.status_code, response.text)
        return None


def get_crypto_price(symbol):
    price_url = 'https://api.api-ninjas.com/v1/cryptoprice?symbol={}'.format(symbol)
    response = requests.get(price_url, headers={'X-Api-Key': ninja_key})
    if response.status_code == requests.codes.ok:
        symbol_ret =json.loads(response.text)['symbol']
        price_ret =json.loads(response.text)['price']
        return symbol_ret, price_ret
    else:
        print("Error:", response.status_code, response.text)
        return None



In [3]:
symbols = get_crypto_symbols()

In [4]:
crypto_prices = pd.DataFrame(columns=['symbol', 'price_USD', 'date'])
crypto_prices['symbol'] = symbols

In [8]:
for row in range(len(crypto_prices)):
    symbol = crypto_prices['symbol'][row]
    symbol_check,price = get_crypto_price(symbol)
    current_date = datetime.now().date()
    if symbol_check == symbol:
        crypto_prices.at[row,'price_USD'] = float(price)
        crypto_prices.at[row, 'date'] = current_date
    else:
        crypto_prices.at[row,'price'] = -1
        crypto_prices.at[row, 'date'] = current_date
        
crypto_prices['price_truc_USD'] = round(crypto_prices['price_USD'])
crypto_prices

,symbol,price_USD,date,price_truc_USD
0,1INCHUSD,0.315,2024-09-17,0.315
1,1INCHUSD4,0.574,2024-09-17,0.574
2,1INCHUSDT,0.2363,2024-09-17,0.2363
3,AAVEUSD,61.3,2024-09-17,61.3
4,AAVEUSD4,85.61,2024-09-17,85.61
...,...,...,...,...
536,ZILUSD4,0.0287,2024-09-17,0.0287
537,ZILUSDT,0.01322,2024-09-17,0.01322
538,ZRXUSD,0.2045,2024-09-17,0.2045
539,ZRXUSD4,0.2525,2024-09-17,0.2525


In [6]:
from sqlalchemy import create_engine

connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

engine = create_engine(connection_string)

In [16]:
try:
    with engine.connect() as connection:
        print("Connection to Redshift successful!")
        # Example query to test connection
        result = connection.execute("SELECT current_date;")
        for row in result:
            print("Current date in Redshift:", row[0])
except Exception as e:
    print(f"Error connecting to Redshift: {e}")

Connection to Redshift successful!
Current date in Redshift: 2024-09-18


In [30]:
query = """
SELECT * from "2024_tomas_fernando_campi_schema".test;
"""

try:
    with engine.connect() as connection:
        print("Connection to Redshift successful!")
        # Example query to test connection
        result = connection.execute(query)
        for row in result:
            print(row)  # This will print each row as a tuple
except Exception as e:
    print(f"Error connecting to Redshift: {e}")



Connection to Redshift successful!
(1, 'Bitcoin', 'BTC', Decimal('21000.00000000'), 396000000000, datetime.datetime(2023, 9, 17, 8, 0))
(2, 'Ethereum', 'ETH', Decimal('1600.00000000'), 190000000000, datetime.datetime(2023, 9, 17, 8, 0))
(3, 'Cardano', 'ADA', Decimal('0.55000000'), 18300000000, datetime.datetime(2023, 9, 17, 8, 0))


In [12]:
crypto_prices.to_sql(name='crypto_prices', con=engine, schema='2024_tomas_fernando_campi_schema', if_exists='append', index=False)


/var/folders/c7/syrz3hl536zfk4dw6xptlb880000gn/T/ipykernel_43778/4228082340.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crypto_prices.to_sql(name='crypto_prices', con=engine, schema='2024_tomas_fernando_campi_schema', if_exists='append', index=False)


AttributeError: 'Engine' object has no attribute 'cursor'

In [ ]:
df_characters = pd.read_csv(csv_path + 'characters.csv')
df_quotes2 = pd.read_sql("SELECT * FROM pda_clases.star_wars_quotes", engine)

pattern = 'Skywalker'

df_skywalker_name = df_characters[df_characters['name'].str.contains(pattern, case=False)]
df_skywalker_desc = df_characters[df_characters['description'].str.contains(pattern, case=False)]

df_skywalker = pd.concat([df_skywalker_name, df_skywalker_desc], ignore_index=False)

df_skywalker = df_skywalker.drop_duplicates()

df_final = pd.merge(df_skywalker, df_quotes2, left_on='name', right_on='character_name')

df_final

In [ ]:
df_resume = df_final[["name", "gender", "homeworld", "quote", "source"]]

df_resume.to_sql(name='characters_resume', con=engine, schema='pda_clases', if_exists='append', index=False)